In [ ]:
import pandas as pd
from datasets import Dataset
import datasets

import evaluate

model_id = 'kreimben/CodeMind-gemma'

perplexity = evaluate.load("perplexity", module_type="metric")

submission_dataset = datasets.load_dataset('kreimben/leetcode_user_submissions_only_python', split='train').to_pandas()
submission_dataset = submission_dataset[['title_slug', 'question_hints', 'question_content', 'content']]
captions_dataset = datasets.load_dataset('kreimben/leetcode_with_youtube_captions', split='train').to_pandas()
captions_dataset = captions_dataset[['title_slug', 'question_hints', 'question_content', 'cc_content']]
captions_dataset.rename(columns={'cc_content': 'content'}, inplace=True)

dataset = pd.concat([submission_dataset, captions_dataset])

del submission_dataset, captions_dataset

dataset = Dataset.from_pandas(dataset)

In [ ]:
GEMMA_2B_IT_MODEL_PREFIX_TEXT = """Below is an coding test problem. Solve the question."""


def generate_prompt(data_point):
    return f"""<start_of_turn>user {GEMMA_2B_IT_MODEL_PREFIX_TEXT}\n
I don't know {data_point['title_slug']} problem. give me the insight or appoach.\n
this is problem's hint.\n{data_point["question_hints"]}\n
here are some content of question.\n{data_point["question_content"]}<end_of_turn>
<start_of_turn>model {data_point["content"]}<end_of_turn>"""

In [ ]:
# add the "prompt" column in the dataset
text_column = [generate_prompt(data_point) for data_point in dataset]
text_column = text_column[:200]
del dataset, GEMMA_2B_IT_MODEL_PREFIX_TEXT

In [ ]:
%%time
results = perplexity.compute(
    model_id=model_id,
    predictions=text_column,
    batch_size=1,
    device='cpu',
)

In [ ]:
print(list(results.keys()))
print(round(results["mean_perplexity"], 2))
print(round(results["perplexities"][0], 2))